In [3]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

https://www.youtube.com/watch?v=eM4uJ6XGnSM


In [4]:
def sigmoid(z):
    return 1/(1+np.e**(-z))

def predict(X, theta):
    z = np.dot(X, theta)
    return sigmoid(z)

def cost_function(X, Y, theta):
    m = len(Y)
    h = predict(X,theta)
    
    cost = (np.dot((-Y.T), np.log(h)) - np.dot((1-Y).T, np.log(1-h))) / m
    return cost

def gradient_descent(X, Y, theta, rate):
    m = len(Y)
    h = predict(X, theta)
    gradient = rate * np.dot(X.T, (h-Y)) / m
    theta -= gradient
    return theta

def decision_boundary(probability):
    classification = 1 if probability >= 0.5 else 0
    return classification

def classify(probabilities):
    return np.vectorize(decision_boundary)(probabilities).flatten()

def train(X, Y, theta, rate, iters):
    cost_history = [2, 1]
    
    for i in range(iters):
        theta = gradient_descent(X, Y, theta, rate)
        
        cost = cost_function(X, Y, theta)
        cost_history.append(cost)
        
        if i % 10000 == 0:
            print("iter: " + str(i) + " cost: "+str(cost))
    return theta, cost_history

def normalize(df, column):
    df[column] = (df[column] -df[column].mean()) / np.std(df[column]) # zscore

In [5]:
passengers = pd.read_csv('titanic_train.csv')
passengers.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [6]:
passengers.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
# convert categorical variables in factors
X = pd.get_dummies(passengers[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']], drop_first=True)
Y = passengers[['Survived']]

# median fill for na values
X['Age'] = X['Age'].fillna(X['Age'].mean())
X['Embarked_S'] = X['Embarked_S'].fillna(X['Embarked_S'].mean())
X['Embarked_Q'] = X['Embarked_Q'].fillna(X['Embarked_Q'].mean())

# normalize values (zscore)
for column in X.columns:
    normalize(X, column)

# add constant for matrix multiplication
constant = np.ones((X.shape[0],1))
X = np.concatenate((constant, X), axis=1)

# initialize theta to zeros vector
theta = np.zeros((X.shape[1],1))

In [8]:
X # design matrix

array([[ 1.00000000e+00,  8.27377244e-01, -5.92480600e-01, ...,
         7.37695132e-01, -3.07562343e-01,  6.19306364e-01],
       [ 1.00000000e+00, -1.56610693e+00,  6.38789012e-01, ...,
        -1.35557354e+00, -3.07562343e-01, -1.61470971e+00],
       [ 1.00000000e+00,  8.27377244e-01, -2.84663197e-01, ...,
        -1.35557354e+00, -3.07562343e-01,  6.19306364e-01],
       ...,
       [ 1.00000000e+00,  8.27377244e-01,  4.37434839e-15, ...,
        -1.35557354e+00, -3.07562343e-01,  6.19306364e-01],
       [ 1.00000000e+00, -1.56610693e+00, -2.84663197e-01, ...,
         7.37695132e-01, -3.07562343e-01, -1.61470971e+00],
       [ 1.00000000e+00,  8.27377244e-01,  1.77062908e-01, ...,
         7.37695132e-01,  3.25137334e+00, -1.61470971e+00]])

In [10]:
trained_theta, cost_history = train(X, Y, theta, 0.0001, 100001)

iter: 0 cost: [[0.45384614]]
iter: 10000 cost: [[0.45156914]]
iter: 20000 cost: [[0.44975291]]
iter: 30000 cost: [[0.44828743]]
iter: 40000 cost: [[0.44709293]]
iter: 50000 cost: [[0.44611051]]
iter: 60000 cost: [[0.44529597]]
iter: 70000 cost: [[0.44461571]]
iter: 80000 cost: [[0.44404384]]
iter: 90000 cost: [[0.44356017]]
iter: 100000 cost: [[0.44314886]]


In [11]:
probabilities = predict(X, trained_theta)
predictions = classify(probabilities)
passengers['prediction'] = predictions

In [12]:
 # percent correct in training
len(passengers[passengers['Survived'] == passengers['prediction']]) / len(passengers)

0.797979797979798

In [16]:
passengers_test = pd.read_csv('titanic_test.csv')

In [17]:
passengers_test.isna().sum()


PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [18]:
# convert categorical variables in factors
X_test = pd.get_dummies(passengers_test[['Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']], drop_first=True)

# median fill for na values
X_test['Age'] = X_test['Age'].fillna(X_test['Age'].mean())
X_test['Fare'] = X_test['Fare'].fillna(X_test['Fare'].mean())

# normalize values (zscore)
for column in X_test.columns:
    normalize(X_test, column)

# add constant for matrix multiplication
constant = np.ones((X_test.shape[0],1))
X_test = np.concatenate((constant, X_test), axis=1)

In [19]:
X_test

array([[ 1.00000000e+00,  8.73481905e-01,  3.34992599e-01, ...,
         7.55928946e-01,  2.84375747e+00, -1.35067551e+00],
       [ 1.00000000e+00,  8.73481905e-01,  1.32553003e+00, ...,
        -1.32287566e+00, -3.51647429e-01,  7.40370278e-01],
       [ 1.00000000e+00, -3.15819190e-01,  2.51417495e+00, ...,
         7.55928946e-01,  2.84375747e+00, -1.35067551e+00],
       ...,
       [ 1.00000000e+00,  8.73481905e-01,  6.51964577e-01, ...,
         7.55928946e-01, -3.51647429e-01,  7.40370278e-01],
       [ 1.00000000e+00,  8.73481905e-01, -2.53374904e-15, ...,
         7.55928946e-01, -3.51647429e-01,  7.40370278e-01],
       [ 1.00000000e+00,  8.73481905e-01, -2.53374904e-15, ...,
         7.55928946e-01, -3.51647429e-01, -1.35067551e+00]])

In [20]:
test_probabilities = predict(X_test, trained_theta)
test_predictions = classify(test_probabilities)
passengers_test['Survived'] = test_predictions

In [21]:
passengers_test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S,0
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q,0
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S,1
